# **Bib**

In [ ]:
import os
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import folium
import matplotlib.pyplot as plt
import random

### `load_plt_file(file_path)`

This function loads a GPS trajectory file in `.plt` format and converts it into a structured Pandas DataFrame.

- **Skips the first 6 header lines** (GeoLife format).
- **Assigns the following column names**:
  - `latitude`: Latitude coordinate
  - `longitude`: Longitude coordinate
  - `zero`: Constant zero (not used)
  - `altitude`: Altitude in meters
  - `days`: Days since a reference date
  - `date`: Date of the GPS point
  - `time`: Time of the GPS point

**Returns:** A DataFrame containing the structured GPS data.

In [ ]:
def load_plt_file(file_path):
    columns = ['latitude', 'longitude', 'zero', 'altitude', 'days', 'date', 'time']
    data = pd.read_csv(file_path, skiprows=6, header=None, names=columns)
    return data

### 📌 `calculate_trajectory_features(data)`

This function extracts three essential features from a GPS trajectory dataset:

- **Total Distance (`km`)**  
  Calculates the total geodesic distance traveled by summing distances between each pair of consecutive points.

- **Total Duration (`hours`)**  
  Computes the time difference between the first and last GPS timestamp.

- **Average Speed (`km/h`)**  
  Derived from:  
  \[
  \text{Average Speed} = \frac{\text{Total Distance}}{\text{Total Duration}}
  \]
  If duration is zero, speed is set to 0.

####  Returns:
A tuple:
- `total_distance`: Total distance in kilometers  
- `total_duration`: Duration in hours  
- `avg_speed`: Average speed in km/h


In [ ]:
def calculate_trajectory_features(data):
    total_distance = 0
    for i in range(1, len(data)):
        point1 = (data.iloc[i-1]['latitude'], data.iloc[i-1]['longitude'])
        point2 = (data.iloc[i]['latitude'], data.iloc[i]['longitude'])
        total_distance += geodesic(point1, point2).km

    start_time = pd.to_datetime(data.iloc[0]['date'] + ' ' + data.iloc[0]['time'])
    end_time = pd.to_datetime(data.iloc[-1]['date'] + ' ' + data.iloc[-1]['time'])
    total_duration = (end_time - start_time).total_seconds() / 3600

    avg_speed = total_distance / total_duration if total_duration > 0 else 0

    return total_distance, total_duration, avg_speed

### 📦 `process_all_trajectories(base_path, max_files_per_user=20)`

This function processes multiple users' GPS trajectory files and extracts features (distance, duration, and speed) from each one.

####  What it does:
- Iterates through all users in the dataset directory.
- Looks into each user's `Trajectory` folder.
- Optionally limits the number of `.plt` files processed per user using `max_files_per_user`.
- For each selected `.plt` file:
  - Loads GPS data using `load_plt_file`.
  - Extracts total distance, duration, and average speed using `calculate_trajectory_features`.
  - Stores the results with `user_id` and file name.

####  Parameters:
- `base_path` (*str*): Path to the main dataset folder containing all users' directories.
- `max_files_per_user` (*int*): Maximum number of trajectory files to process per user (default = 20).

####  Returns:
A Pandas DataFrame with the following columns:
- `user_id`
- `file`
- `distance` (km)
- `duration` (hours)
- `speed` (km/h)


def process_all_trajectories(base_path, max_files_per_user=20):
    all_features = []
    for user_id in os.listdir(base_path):
        user_path = os.path.join(base_path, user_id, "Trajectory")
        if os.path.exists(user_path):
            plt_files = os.listdir(user_path)
            if len(plt_files) > max_files_per_user:
                plt_files = random.sample(plt_files, max_files_per_user)

            for plt_file in plt_files:
                plt_path = os.path.join(user_path, plt_file)
                try:
                    data = load_plt_file(plt_path)
                    distance, duration, speed = calculate_trajectory_features(data)
                    all_features.append({
                        'user_id': user_id,
                        'file': plt_file,
                        'distance': distance,
                        'duration': duration,
                        'speed': speed
                    })
                except Exception as e:
                    print(f"Error processing {plt_file}: {e}")
    return pd.DataFrame(all_features)


### 📥 Installing and Using the Kaggle API to Download the GeoLife Dataset

This section demonstrates how to install the Kaggle API, authenticate using your Kaggle API key, and download the GeoLife dataset.

#### 🔧 Steps:
1. **Install the Kaggle API:**
   The `kaggle` Python package is installed to allow interaction with Kaggle datasets programmatically.
   ```python
   !pip install kaggle


In [ ]:

!pip install kaggle


import os
os.makedirs("/root/.kaggle", exist_ok=True)


from google.colab import drive
drive.mount('/content/drive')


!cp /content/drive/MyDrive/kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d marquis03/geolife-trajectories-1-3
!unzip geolife-trajectories-1-3.zip -d ./geolife_data


### 🛣️ Optimal Trajectory Evaluation and Visualization

This code processes trajectory data to evaluate different routes based on distance, duration, and speed, and calculates the utility of each route to determine the optimal one.

#### 🔧 Steps:

1. **Set the Base Path for Data:**
   - The `base_path` variable is set to the location of the trajectory data.
   ```python
   base_path = "path to the data"


In [ ]:


base_path = "path to the data"


evaluation_matrix = process_all_trajectories(base_path, max_files_per_user=5)
print(evaluation_matrix.head())


normalized_matrix = evaluation_matrix.copy()
normalized_matrix['distance'] /= normalized_matrix['distance'].sum()
normalized_matrix['duration'] = (normalized_matrix['duration'].max() - normalized_matrix['duration']) / \
                                (normalized_matrix['duration'].max() - normalized_matrix['duration'].min())
normalized_matrix['speed'] /= normalized_matrix['speed'].sum()
print(normalized_matrix.head())

proportions = normalized_matrix[['distance', 'duration', 'speed']].div(normalized_matrix[['distance', 'duration', 'speed']].sum(axis=0))
entropy = -np.sum(proportions * np.log(proportions + 1e-12), axis=0) / np.log(len(proportions))
variation_coefficients = 1 - entropy
objective_weights = variation_coefficients / variation_coefficients.sum()
print("Objective Weights:", objective_weights)


subjective_weights = np.array([0.4, 0.3, 0.3])
alpha = 0.5
global_weights = alpha * subjective_weights + (1 - alpha) * objective_weights
print("Global Weights:", global_weights)


global_utility = np.dot(normalized_matrix[['distance', 'duration', 'speed']], global_weights)
evaluation_matrix['Utility'] = global_utility


optimal_trajectory = evaluation_matrix.loc[evaluation_matrix['Utility'].idxmax()]
print("Optimal Trajectoiry :", optimal_trajectory)


plt.figure(figsize=(10, 6))
plt.bar(evaluation_matrix.index, evaluation_matrix['Utility'], color='skyblue')
plt.xlabel("Trajectoires", fontsize=12)
plt.ylabel("Utility", fontsize=12)
plt.title("Trajectories Evaluation - Utility Scores", fontsize=14)
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


optimal_file_path = os.path.join(base_path, optimal_trajectory['user_id'], 'Trajectory', optimal_trajectory['file'])
optimal_data = load_plt_file(optimal_file_path)

map_center = [optimal_data.iloc[0]['latitude'], optimal_data.iloc[0]['longitude']]
m = folium.Map(location=map_center, zoom_start=12)

for i in range(1, len(optimal_data)):
    point1 = [optimal_data.iloc[i-1]['latitude'], optimal_data.iloc[i-1]['longitude']]
    point2 = [optimal_data.iloc[i]['latitude'], optimal_data.iloc[i]['longitude']]
    folium.PolyLine([point1, point2], color='blue', weight=2.5, opacity=1).add_to(m)

m.save("optimal_trajectory_map.html")

